In [ ]:
import json

class Student:
    def __init__(self, roll_number, name, password, class_name, email):
        self.roll_number = roll_number
        self.name = name
        self.password = password
        self.class_name = class_name
        self.email = email

class GatePass:
    def __init__(self, roll_number, name, class_name, reason):
        self.roll_number = roll_number
        self.name = name
        self.class_name = class_name
        self.reason = reason
        self.status = "Pending"

class Admin:
    def __init__(self, username, password):
        self.username = username
        self.password = password

class GatePassManager:
    def __init__(self):
        self.students = {}
        self.gate_pass_requests = []
        self.admins = {}
        self.load_data()

    def load_data(self):
        try:
            with open('students.json', 'r') as file:
                content = file.read().strip()
                self.students = json.loads(content) if content else {}
        except (FileNotFoundError, json.JSONDecodeError):
            self.students = {}
    
        try:
            with open('gate_pass_requests.json', 'r') as file:
                content = file.read().strip()
                self.gate_pass_requests = json.loads(content) if content else []
        except (FileNotFoundError, json.JSONDecodeError):
            self.gate_pass_requests = []
    
        try:
            with open('admins.json', 'r') as file:
                content = file.read().strip()
                self.admins = json.loads(content) if content else {}
        except (FileNotFoundError, json.JSONDecodeError):
            self.admins = {}

    def save_data(self):
        with open('students.json', 'w') as file:
            json.dump(self.students, file, indent=4)
        with open('gate_pass_requests.json', 'w') as file:
            json.dump(self.gate_pass_requests, file, indent=4)
        with open('admins.json', 'w') as file:
            json.dump(self.admins, file, indent=4)

    def register_student(self, roll_number, name, password, class_name, email):
        if roll_number in self.students:
            return False
        self.students[roll_number] = {
            'name': name,
            'password': password,
            'class': class_name,
            'email': email  # ✅ Fixed: email is now correctly included
        }
        self.save_data()
        return True

    def login_student(self, roll_number, password):
        student = self.students.get(roll_number)
        if student and student['password'] == password:
            return student['name'], student['class']
        return None, None

    def apply_gate_pass(self, roll_number, name, class_name, reason):
        gate_pass = GatePass(roll_number, name, class_name, reason)
        self.gate_pass_requests.append(gate_pass.__dict__)
        self.save_data()

    def check_gate_pass_status(self, roll_number):
        return [gp for gp in self.gate_pass_requests if gp['roll_number'] == roll_number]

    def withdraw_gate_pass(self, roll_number):
        for request in self.gate_pass_requests:
            if request['roll_number'] == roll_number:
                self.gate_pass_requests.remove(request)
                self.save_data()
                return True
        return False

    def approve_gate_pass(self, roll_number):
        for request in self.gate_pass_requests:
            if request['roll_number'] == roll_number:
                request['status'] = 'Approved'
        self.save_data()

    def reject_gate_pass(self, roll_number):
        for request in self.gate_pass_requests:
            if request['roll_number'] == roll_number:
                request['status'] = 'Rejected'
        self.save_data()

    def register_admin(self, username, password):
        if username in self.admins:
            return False
        self.admins[username] = password
        self.save_data()
        return True

    def login_admin(self, username, password):
        return self.admins.get(username) == password


def main():
    manager = GatePassManager()
    
    while True:
        print("\nOptions:")
        print("1. Register Student")
        print("2. Login Student")
        print("3. Search for gate pass with roll number")
        print("4. Register Admin")
        print("5. Login Admin")
        print("6. Exit")
        choice = input("Choose an option: ")

        if choice == '1':
            roll_number = input("Enter roll number: ")
            name = input("Enter name: ")
            password = input("Enter password: ")
            class_name = input("Enter class: ")
            email = input("Enter email: ")  # ✅ Fixed: Ask for email input
            if manager.register_student(roll_number, name, password, class_name, email):
                print("Registration successful!")
            else:
                print("Roll number already exists.")

        elif choice == '2':
            roll_number = input("Enter roll number: ")
            password = input("Enter password: ")
            name, class_name = manager.login_student(roll_number, password)
            if name:
                print(f"Welcome {name}! You are in class {class_name}.")
                while True:
                    print("\nOptions:")
                    print("1. Apply for gate pass")
                    print("2. Check status of gate pass")
                    print("3. Withdraw gate pass")
                    print("4. Logout")
                    sub_choice = input("Choose an option: ")

                    if sub_choice == '1':
                        reason = input("Enter reason for gate pass: ")
                        manager.apply_gate_pass(roll_number, name, class_name, reason)
                        print("Gate pass request submitted.")

                    elif sub_choice == '2':
                        statuses = manager.check_gate_pass_status(roll_number)
                        if statuses:
                            for status in statuses:
                                print(f"Gate Pass for {status['reason']}: {status['status']}")
                        else:
                            print("No gate pass requests found.")

                    elif sub_choice == '3':
                        manager.withdraw_gate_pass(roll_number)
                        print("Gate pass withdrawn.")

                    elif sub_choice == '4':
                        print("Logged out.")
                        break

                    else:
                        print("Invalid option.")

            else:
                print("Invalid roll number or password.")

        elif choice == '3':
            roll_number = input("Enter roll number to search: ")
            statuses = manager.check_gate_pass_status(roll_number)
            if statuses:
                for status in statuses:
                    print(f"Gate Pass for {status['reason']}: {status['status']}")
            else:
                print("No gate pass requests found.")

        elif choice == '4':
            username = input("Enter admin username: ")
            password = input("Enter admin password: ")
            if manager.register_admin(username, password):
                print("Admin registration successful!")
            else:
                print("Admin username already exists.")

        elif choice == '5':
            username = input("Enter admin username: ")
            password = input("Enter admin password: ")
            if manager.login_admin(username, password):
                print(f"Welcome Admin {username}!")
                while True:
                    print("\nOptions:")
                    print("1. View all gate passes")
                    print("2. Approve gate pass")
                    print("3. Reject gate pass")
                    print("4. Logout")
                    admin_choice = input("Choose an option: ")

                    if admin_choice == '1':
                        for request in manager.gate_pass_requests:
                            print(f"{request['name']} ({request['roll_number']}): {request['reason']} - {request['status']}")

                    elif admin_choice == '2':
                        roll_number = input("Enter roll number to approve: ")
                        manager.approve_gate_pass(roll_number)
                        print("Gate pass approved.")

                    elif admin_choice == '3':
                        roll_number = input("Enter roll number to reject: ")
                        manager.reject_gate_pass(roll_number)
                        print("Gate pass rejected.")

                    elif admin_choice == '4':
                        print("Logged out.")
                        break

                    else:
                        print("Invalid option.")
            
            else:
                print("Invalid admin username or password.")

        elif choice == '6':
            print("Exiting the program.")
            break

        else:
            print("Invalid option.")

if __name__ == "__main__":
    main()



Options:
1. Register Student
2. Login Student
3. Search for gate pass with roll number
4. Register Admin
5. Login Admin
6. Exit


Choose an option:  1
Enter roll number:  kiran
Enter name:  kiran
Enter password:  kiran
Enter class:  aiml_c
Enter email:  kiran@gmail.com


Registration successful!

Options:
1. Register Student
2. Login Student
3. Search for gate pass with roll number
4. Register Admin
5. Login Admin
6. Exit


Choose an option:  3
Enter roll number to search:  kiran


No gate pass requests found.

Options:
1. Register Student
2. Login Student
3. Search for gate pass with roll number
4. Register Admin
5. Login Admin
6. Exit


...

Kousalya (160121733005): Dental Appointment - Approved

Options:
1. View all gate passes
2. Approve gate pass
3. Reject gate pass
4. Logout


Choose an option:  4


Logged out.

Options:
1. Register Student
2. Login Student
3. Search for gate pass with roll number
4. Register Admin
5. Login Admin
6. Exit


Choose an option:  6


Exiting the program.


In [ ]:
import json

def read_students_data():
    try:
        with open('students.json', 'r') as file:
            students = json.load(file)
            print("Students Data:")
            print(json.dumps(students, indent=4))
    except FileNotFoundError:
        print("students.json file not found.")
    except json.JSONDecodeError:
        print("Error decoding JSON from students.json.")

# Call the function
read_students_data()


In [ ]:
import json

def read_admins_data():
    try:
        with open('admins.json', 'r') as file:
            admins = json.load(file)
            print("Admins Data:")
            print(json.dumps(admins, indent=4))
    except FileNotFoundError:
        print("admins.json file not found.")
    except json.JSONDecodeError:
        print("Error decoding JSON from admins.json.")

# Call the function
read_admins_data()


In [ ]:
import json

def read_gate_pass_requests_data():
    try:
        with open('gate_pass_requests.json', 'r') as file:
            gate_pass_requests = json.load(file)
            print("Gate Pass Requests Data:")
            print(json.dumps(gate_pass_requests, indent=4))
    except FileNotFoundError:
        print("gate_pass_requests.json file not found.")
    except json.JSONDecodeError:
        print("Error decoding JSON from gate_pass_requests.json.")

# Call the function
read_gate_pass_requests_data()


# 